<a target="_blank" href="https://colab.research.google.com/github/UpstageAI/cookbook/blob/main/Solar-Full-Stack LLM-101/05_3_OracleDB.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# 05-3. LLMChain_OracleDB
## Overview  
In this exercise, we will explore how to leverage OracleDB to embed documents and create a vectorspace. You will learn how to develop a Retriever object and perform document searches efficiently. Furthermore, this tutorial will guide you through creating an LLM Chain integrated with a Retriever object, enhancing the capabilities of your search and retrieval processes.
 
## Purpose of the Exercise
The purpose of this exercise is to demonstrate how to integrate OracleDB with Python, utilize the Solar Embedding API to generate embeddings and create a vectorspace, and create a Retriever object to conduct searches within the vectorspace. By the end of this tutorial, users will also understand how to create an LLM Chain combined with a Retriever object, thereby improving the efficiency and accuracy of document retrieval using OracleDB.


## OracleVectorDB
- benefits?
- where to get KEYs and how?

In [1]:
!pip3 install -qU python-dotenv PyPDF2 langchain langchain-community langchain-core langchain-text-splitters langchain_upstage oracledb python-dotenv


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [1]:
# @title set API key
from pprint import pprint
import os

import warnings

warnings.filterwarnings("ignore")

from IPython import get_ipython

upstage_api_key_env_name = "UPSTAGE_API_KEY"
oracle_db_user_env_name = "ORACLE_DB_USER"
oracle_db_password_env_name = "ORACLE_DB_PASSWORD"
oracle_dsn_env_name = "ORACLE_DSN"


def load_env():
    if "google.colab" in str(get_ipython()):
        # Running in Google Colab
        from google.colab import userdata

        upstage_api_key = userdata.get(upstage_api_key_env_name)
        oracle_db_user = userdata.get(oracle_db_user_env_name)
        oracle_db_password = userdata.get(oracle_db_password_env_name)
        oracle_dsn = userdata.get(oracle_dsn_env_name)
        return (
            os.environ.setdefault(upstage_api_key_env_name, upstage_api_key),
            os.environ.setdefault(oracle_db_user_env_name, oracle_db_user),
            os.environ.setdefault(oracle_db_password_env_name, oracle_db_password),
            os.environ.setdefault(oracle_dsn_env_name, oracle_dsn),
        )
    else:
        # Running in local Jupyter Notebook
        from dotenv import load_dotenv

        load_dotenv()
        return (
            os.environ.get(upstage_api_key_env_name),
            os.environ.get(oracle_db_user_env_name),
            os.environ.get(oracle_db_password_env_name),
            os.environ.get(oracle_dsn_env_name),
        )


UPSTAGE_API_KEY, ORACLE_DB_USER, ORACLE_DB_PASSWORD, ORACLE_DSN = load_env()

In [5]:
import sys
import array
import time
import os
from dotenv import load_dotenv

import oracledb
from langchain_community.vectorstores import oraclevs
from langchain_community.vectorstores.oraclevs import OracleVS

from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_core.documents import BaseDocumentTransformer, Document

from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [8]:
try:
    conn23c = oracledb.connect(
        user=os.environ["ORACLE_DB_USER"],
        password=os.environ["ORACLE_DB_PASSWORD"],
        dsn=os.environ["ORACLE_DSN"],
    )
    print("Connection successful!", conn23c.version)
except Exception as e:
    print("Connection failed!", e)

Connection successful! 23.4.1.24.6


In [9]:
from langchain_upstage import UpstageLayoutAnalysisLoader


layzer = UpstageLayoutAnalysisLoader("pdfs/kim-tse-2008.pdf", output_type="html")
# For improved memory efficiency, consider using the lazy_load method to load documents page by page.
docs = layzer.load()  # or layzer.lazy_load()

In [13]:
from langchain_text_splitters import (
    Language,
    RecursiveCharacterTextSplitter,
)

# 2. Split
text_splitter = RecursiveCharacterTextSplitter.from_language(
    chunk_size=1000, chunk_overlap=100, language=Language.HTML
)
splits = text_splitter.split_documents(docs)
print("Splits:", len(splits))

Splits: 132


In [14]:
from langchain_upstage import UpstageEmbeddings

upstage_embeddings = UpstageEmbeddings(model="solar-embedding-1-large")
knowledge_base = OracleVS.from_documents(
    splits,
    upstage_embeddings,
    client=conn23c,
    table_name="text_embeddings2",
    distance_strategy=DistanceStrategy.DOT_PRODUCT,
)

In [16]:
vector_store = OracleVS(
    client=conn23c,
    embedding_function=upstage_embeddings,
    table_name="text_embeddings2",
    distance_strategy=DistanceStrategy.DOT_PRODUCT,
)
retriever = vector_store.as_retriever()

In [29]:
# Query the retriever
result_docs = retriever.invoke("What is change classficiation?")
print("Results:", len(result_docs))
print(result_docs[0].page_content[:100])

Results: 4
<p id='56' data-category='paragraph' style='font-size:16px'>Change classification differs from previ


In [30]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_upstage import ChatUpstage


llm = ChatUpstage()

prompt_template = PromptTemplate.from_template(
    """
    Please provide most correct answer from the following context. 
    If the answer is not present in the context, please write "The information is not present in the context."
    ---
    Question: {question}
    ---
    Context: {context}
    """
)
chain = prompt_template | llm | StrOutputParser()

In [31]:
chain.invoke({"question": "What is change classficiation?", "context": result_docs})

'Change classification is a process that differs from previous bug prediction work. It classifies changes and focuses on predicting whether there is a bug in any of the lines that were changed in one file in one SCM commit transaction. Unlike bug prediction work, change classification uses features from the source code, such as every variable, method call, operator, constant, comment text, and more, to train change classification models. It is also independent of the programming language and uses bug-introducing changes to label changes as buggy or clean with information about the source code at the moment a bug was introduced.'

In [33]:
# Put themm all together
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)

rag_chain = setup_and_retrieval | prompt_template | llm | StrOutputParser()

rag_chain.invoke("What is change classification?")

'Change classification is a process that differs from previous bug prediction work in that it classifies changes, predicts whether there is a bug in any of the lines that were changed in one file in one SCM commit transaction, and uses every term in the source code as features to train change classification models. It is also independent of the programming language and uses bug-introducing changes.'